In [1]:
!pip install mlrose_hiive
import time
import sklearn
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('figure', figsize=[10,5])
import mlrose_hiive
from sklearn.metrics import f1_score
from functools import partial
from google.colab import drive
from sklearn import preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [2]:
np.random.seed(44)
train_data = pd.read_csv('sample_data/mnist_train_small.csv')
test_data = pd.read_csv('sample_data/mnist_test.csv')

In [3]:
train_y = train_data['6']
train_y.columns = ['label']
test_y = test_data['7']
test_y.columns = ['label']
train_X = train_data.drop(columns = ['6'])
test_X = test_data.drop(columns = ['7'])

feature_names = []
for i in range(len(train_X.columns)):
    feature_names.append(i)

train_X.columns = feature_names
test_X.columns = feature_names 

In [4]:
start_time = time.time()

end_time = time.time()
run_time = end_time - start_time


In [ ]:
%%time
start_time = time.time()
f1_labs = partial(f1_score, average="weighted")
grid_search_parameters = ({
  'learning_rate': [0.01, 0.001],
  'restarts': [5, 25, 50],
  'activation': [mlrose_hiive.neural.activation.relu]
})
rhc_nn = mlrose_hiive.NNGSRunner(train_X, pd.get_dummies(train_y.values.ravel()).values, 
                                 test_X, pd.get_dummies(test_y.values.ravel()).values, 
                                 "NeuralNetworkRHC",
                                 output_directory="./",
                                 seed=10, iteration_list=[1000], 
                                 algorithm=mlrose_hiive.random_hill_climb,
                                 hidden_layer_sizes=[[60,60]],
                                 grid_search_parameters=grid_search_parameters,
                                 grid_search_scorer_method=f1_labs,
                                 n_jobs=-2, cv=5)
result = rhc_nn.run()

end_time = time.time()
run_time = end_time - start_time

print("RHC Run time: " + str(run_time))

Running nngs_rhc
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.


*****************
*** Run START ***
*****************
max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ...  0.98872842  0.15632159
  0.99258313]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[0]
runner_name:[nngs_rhc], experiment_name:[NeuralNetworkRHC], attempt:[0], iteration:[0], done:[False], time:[0.32], fitness:[31.3781]
	[ 0.54264129 -0.9584961   0.26729647 ...  0.98872842  0.15632159//  0.99258313]...

max_iters:[100], learning_rate:[0.01], restarts:[5], init_state:[[ 0.54264129 -0.9584961   0.26729647 ...  0.98872842  0.15632159
  0.99258313]], algorithm:[rhc], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500], current_restart:[0]
runner_name:[nngs_rhc], experiment_name:[Neura

In [ ]:
y_pred = result[3].predict(test_X)
print(classification_report(pd.get_dummies(test_y.values.ravel()).values, y_pred))

In [ ]:
y_pred_train = result[3].predict(train_X)
print(classification_report(pd.get_dummies(train_y.values.ravel()).values, y_pred_train))